# FastAPI Requests

This notebook demonstrates, how requests can be submitted to the running FastAPI.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json
import requests

## HTML Data Loading

Load data for the API requests:

In [3]:
with open('../html_body.txt', 'r', encoding='utf-8') as file:
    raw_html = file.read()

## Health Check

If not done before, make sure that the API is up and running for the next steps:

```sh
uvicorn src.api.app:app --reload
```

Confirm that the API is running via the `/health` endpoint:

In [5]:
requests.get('http://127.0.0.1:8000/health').json()

'Comment API is healthy'

## Request Submission

Let's submit actual requests to the API.

### Comment Extraction

Pure extraction of comments:

In [6]:
response = requests.post(
    url='http://127.0.0.1:8000/extract-comments',
    json={
        "raw_html": raw_html
    }
)

In [7]:
response.status_code

200

Assuming a successful request (status code 200), we can yield the comments from the response as shown below:

In [8]:
comments = response.json()['comments']

In [9]:
print(f"{len(comments)} comments extracted.")

225 comments extracted.


### Complete Comment Analysis

Analysis of the complete raw HTML including comments, donations, and dislike estimation:

In [10]:
response = requests.post(
    url='http://127.0.0.1:8000/analyze-comments',
    json={
        "raw_html": raw_html,
        "params": {
            "analyze_donations": True,
            "target_currency": "USD",
            "dislike_polarity_cutoff": 0.3
        }
    }
)

In [11]:
response.status_code

200

Assuming a successful request (status code 200), we can check the metrics of the analysis below:

In [12]:
print(json.dumps(response.json(), indent=3))

{
   "comments": {
      "raw_comments__count": 225,
      "translated_comments__count": 221,
      "sentiment_comments__vader__median": 0.6369,
      "sentiment_comments__textblob__median": 0.2
   },
   "donations": {
      "donations__count": 80,
      "donations__sum": 1826.49,
      "donations__currency": "USD"
   },
   "dislikes": {
      "highly_positive_comments__count": 140,
      "highly_negative_comments__count": 3,
      "positive_to_negative_comment_ratio": 46.666666666666664,
      "likes__count": 314865,
      "dislikes__estimate": 6747
   }
}
